## Connection

In [1]:
from pony.orm import * 

db = Database()
db_path = '/home/bacek/lessons/python/fast_api/examples/example_5_pony/database/database.sqlite'

class User(db.Entity):
    user_id = Required(int)
    nick = Required(str)
    age = Required(int)
    wallets = Set('Wallet')

class Wallet(db.Entity):
    address = Required(str)
    private_key = Required(str)
    owner = Required(User)


try: 
    db.bind(provider='sqlite', filename=db_path, create_db=True)
    db.generate_mapping(create_tables=True)
except Exception as e:
    print(e)

## Debug

In [2]:
# Включения режима debug
set_sql_debug()

## Insert

In [42]:
u1 = User(user_id=13,nick='mangust228',age=33)
u2 = User(user_id=7,nick='hello',age=35)
w1 = Wallet(address='moscow', private_key='AS32KJFD', owner=u2)
w2 = Wallet(address='samui', private_key='IVCXREW32DT', owner=u1)
commit()

BEGIN IMMEDIATE TRANSACTION
INSERT INTO "User" ("user_id", "nick", "age") VALUES (?, ?, ?)
[13, 'mangust228', 33]

INSERT INTO "User" ("user_id", "nick", "age") VALUES (?, ?, ?)
[7, 'hello', 35]

INSERT INTO "Wallet" ("address", "private_key", "owner") VALUES (?, ?, ?)
['moscow', 'AS32KJFD', 4]

INSERT INTO "Wallet" ("address", "private_key", "owner") VALUES (?, ?, ?)
['samui', 'IVCXREW32DT', 3]

COMMIT


## Select

### Select * from ....

In [3]:
User.select().show()

GET NEW CONNECTION
SWITCH TO AUTOCOMMIT MODE
SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"

id|user_id|nick      |age
--+-------+----------+---
1 |20     |John      |25 
2 |20     |Mary      |33 
3 |13     |mangust228|33 
4 |7      |hello     |35 
5 |3      |Vasiliy   |25 


In [4]:
# Вариант 2
select(u for u in User).show()

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"

id|user_id|nick      |age
--+-------+----------+---
1 |20     |John      |25 
2 |20     |Mary      |33 
3 |13     |mangust228|33 
4 |7      |hello     |35 
5 |3      |Vasiliy   |25 


In [5]:
# Вариант 3
select(u for u in User)[:]

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"



[User[1], User[2], User[3], User[4], User[5]]

### Select where 

In [6]:
select(u for u in User if u.age > 33).show()

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"
WHERE "u"."age" > 33

id|user_id|nick |age
--+-------+-----+---
4 |7      |hello|35 


In [7]:
select(u for u in User if 'o' in u.nick)[:]

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"
WHERE "u"."nick" LIKE '%o%'



[User[1], User[4]]

In [8]:
u1 = User[1]
print(u1.nick)

John


In [9]:
mary = User.get(nick='Mary')

SELECT "id", "user_id", "nick", "age"
FROM "User"
WHERE "nick" = ?
LIMIT 2
['Mary']



## Limit/Order by

In [10]:
# Order
select(u for u in User).order_by(User.nick)[:]

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"
ORDER BY "u"."nick"



[User[1], User[2], User[5], User[4], User[3]]

In [11]:
# Limit
select(u for u in User)[:2]

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"
LIMIT 2



[User[1], User[2]]

In [12]:
# Offset
select(u for u in User)[1:3]

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"
LIMIT 2 OFFSET 1



[User[2], User[3]]

## UPDATE

In [14]:
mary = User.get(nick='Mary')

SELECT "id", "user_id", "nick", "age"
FROM "User"
WHERE "nick" = ?
LIMIT 2
['Mary']



In [16]:
mary.age += 1
commit()

BEGIN IMMEDIATE TRANSACTION
UPDATE "User"
SET "age" = ?
WHERE "id" = ?
  AND "user_id" = ?
  AND "nick" = ?
  AND "age" = ?
[34, 2, 20, 'Mary', 33]

COMMIT


## Работа по SQL-запросам

In [17]:
x = 30
User.select_by_sql('Select * FROM User u WHERE u.age < $x')

BEGIN IMMEDIATE TRANSACTION
Select * FROM User u WHERE u.age < ?
[30]



[User[1], User[5]]

### Минуя сущности

In [21]:
z = 28
db.select('nick FROM User WHERE age > $z')

select nick FROM User WHERE age > ?
[28]



['Mary', 'mangust228', 'hello']

## Дополнительные методы

### Filter

In [28]:
query = select(u for u in User)
query_new = query.filter(lambda person: person.age > 18)
query_new.show()

SELECT "u"."id", "u"."user_id", "u"."nick", "u"."age"
FROM "User" "u"
WHERE "u"."age" > 18

id|user_id|nick      |age
--+-------+----------+---
1 |20     |John      |25 
2 |20     |Mary      |34 
3 |13     |mangust228|33 
4 |7      |hello     |35 
5 |3      |Vasiliy   |25 


In [30]:
# Вернуть первый из списка
query_new.first().id

1

## Пример из документации

In [33]:
from pony.orm.examples.estore import * 

GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
CREATE TABLE "Category" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT UNIQUE NOT NULL
)

CREATE TABLE "Customer" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "email" TEXT UNIQUE NOT NULL,
  "password" TEXT NOT NULL,
  "name" TEXT NOT NULL,
  "country" TEXT NOT NULL,
  "address" TEXT NOT NULL
)

CREATE TABLE "Order" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "state" TEXT NOT NULL,
  "date_created" DATETIME NOT NULL,
  "date_shipped" DATETIME,
  "date_delivered" DATETIME,
  "total_price" DECIMAL(12, 2) NOT NULL,
  "customer" INTEGER NOT NULL REFERENCES "Customer" ("id") ON DELETE CASCADE
)

CREATE INDEX "idx_order__customer" ON "Order" ("customer")

CREATE TABLE "Product" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "description" TEXT NOT NULL,
  "picture" BLOB,
  "price" DECIMAL(12, 2) NOT NULL,
  "quantity" INTEGER NOT

![image](https://ucarecdn.com/614300f1-8be0-485f-8175-9d5feaa841fe/)

In [46]:
[(i,j)for i in range(4) 
      for j in range(4,6)]

[(0, 4), (0, 5), (1, 4), (1, 5), (2, 4), (2, 5), (3, 4), (3, 5)]

In [47]:
# Все клиенты из США
Customer.select(lambda c: c.country == 'USA')

# Количество клиентов для каждой страны
select((c.country, count(c)) for c in Customer)

# Максимальная цена продукта
max(p.price for p in Product)

# Максимальная цена SSD
max(p.price for p in Product
            for cat in p.categories if cat.name == 'Solid State Drives')

# Три самых дорогих продукта
Product.select().order_by(desc(Product.price))[:3]

# Продукты, отсутствующие на складе
Product.select(lambda p: p.quantity == 0)

# Самый популярный товар
Product.select().order_by(lambda p: desc(sum(p.order_items.quantity))).first()

# Продукты, которые никогда не были заказаны
Product.select(lambda p: not p.order_items)

# Клиенты, которые сделали несколько заказов
Customer.select(lambda c: count(c.orders) > 1)

# Три самых ценных клиента
Customer.select().order_by(lambda c: desc(sum(c.orders.total_price)))[:3]

# Клиенты, чьи заказы были отправлены
Customer.select(lambda c: SHIPPED in c.orders.state)

# Клиенты, у которых нет заказов
Customer.select(lambda c: not c.orders)

# Тот же запрос с LEFT JOIN вместо NOT EXISTS
left_join(c for c in Customer for o in c.orders if o is None)

# Клиенты, которые заказали несколько разных планшетов
select(c for c in Customer
         for p in c.orders.items.product
         if 'Tablets' in p.categories.name and count(p) > 1)

SELECT MAX("p"."price")
FROM "Product" "p"

SELECT MAX("p"."price")
FROM "Product" "p", "Category_Product" "t-1", "Category" "cat"
WHERE "cat"."name" = 'Solid State Drives'
  AND "p"."id" = "t-1"."product"
  AND "t-1"."category" = "cat"."id"

SELECT "p"."id", "p"."name", "p"."description", "p"."picture", "p"."price", "p"."quantity"
FROM "Product" "p"
ORDER BY "p"."price" DESC
LIMIT 3

SELECT "p"."id"
FROM "Product" "p"
  LEFT JOIN "OrderItem" "orderitem"
    ON "p"."id" = "orderitem"."product"
GROUP BY "p"."id"
ORDER BY coalesce(SUM("orderitem"."quantity"), 0) DESC
LIMIT 1

SELECT "c"."id"
FROM "Customer" "c"
  LEFT JOIN "Order" "order"
    ON "c"."id" = "order"."customer"
GROUP BY "c"."id"
ORDER BY coalesce(SUM("order"."total_price"), 0) DESC
LIMIT 3



In [36]:
select((c.country, count(c)) for c in Customer)[:]


GET NEW CONNECTION
SWITCH TO AUTOCOMMIT MODE
SELECT "c"."country", COUNT(DISTINCT "c"."id")
FROM "Customer" "c"
GROUP BY "c"."country"



[]

## Использование даты и времени

In [55]:
from datetime import datetime, timedelta

In [56]:
select(o for o in Order if o.date_created >= datetime.now() - timedelta(days=3))[:]

SELECT "o"."id", "o"."state", "o"."date_created", "o"."date_shipped", "o"."date_delivered", "o"."total_price", "o"."customer"
FROM "Order" "o"
WHERE "o"."date_created" >= ?
['2023-01-12 11:21:56.268564']



[]

## DISTINCT 

Pony добавляет ключевое слово DISCTINCT только в тех случаях, когда возможны дубликаты. Рассмотрим несколько примеров.

In [64]:
# Не добавит
Customer.select(lambda p: p.name == 'John')[:]


SELECT "p"."id", "p"."email", "p"."password", "p"."name", "p"."country", "p"."address"
FROM "Customer" "p"
WHERE "p"."name" = 'John'



[]

In [65]:
# Добавит
select(c.name for c in Customer)[:]

SELECT DISTINCT "c"."name"
FROM "Customer" "c"



[]

In [68]:
# Чтоб не добавлял:
select(c.name for c in Customer).without_distinct()[:]

SELECT "c"."name"
FROM "Customer" "c"



[]

## Функции внутри запроса

- `avg()` 
- `abs()` 
- `exists()` 
- `len()` 
- `max()` 
- `min()` 
- `count()` 
- `concat()` 
- `group_concat()` 
- `random()` 
- `raw_sql()` 
- `select()` 
- `sum()` 
- `getattr()` 
- `startswith()` 